In [101]:
from os import listdir
from os.path import join
import os.path

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt

import random

In [102]:
import gdown
import patoolib

# google drive link
url       = 'https://drive.google.com/uc?id='
file_id   = '1iVt97n4fpFKUeTW93kNGMzbadRbHCjUf'

# save path
data_path = '../data/'
data_dir  = 'extracted_images'
data_file = 'math_data.rar'

if os.path.isdir(data_path+data_dir):
    print('already exist')

# file download
elif not os.path.isfile(data_path+data_file):
    gdown.download(url + file_id, data_path+data_file, quiet=False)
    patoolib.extract_archive(data_path+data_file, outdir='../data/')
    
else:
    patoolib.extract_archive(data_path+data_file, outdir='../data/')


already exist


In [103]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

training_epochs = 25
batch_size = 256

using cuda


In [104]:
# image preprocessing
# grayscale, 
# option: resize
trans = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor() 
    ])
trainset = torchvision.datasets.ImageFolder(root = data_path + 'extracted_images',
                                            transform = trans)
classes = trainset.classes
len(trainset.classes)

print(trainset.classes[63])
print(len(trainset.targets))

pi
375974


In [105]:
train_size = int(0.9 * len(trainset))
test_size = len(trainset) - train_size

train_data, test_data = torch.utils.data.random_split(trainset, [train_size, test_size])

train_loader = DataLoader(train_data,
                         batch_size = batch_size,
                         shuffle = True,
                         drop_last = True)

test_loader = DataLoader(test_data,
                         batch_size = batch_size,
                         shuffle = False,
                         drop_last = True)

print(f'train size = {train_size}\ntest size = {test_size}')
print('')
print(f'training data set = {len(train_loader)}\ntest data set = {len(test_loader)}')


images, labels = next(iter(train_loader))
images.shape, labels.shape

train size = 338376
test size = 37598

training data set = 1321
test data set = 146


(torch.Size([256, 1, 45, 45]), torch.Size([256]))

In [120]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # output = ( input - filter(kernel) + 2 * padding) / stride ) + 1
        self.conv1 = nn.Conv2d(1, 64, 3, 1) # input, output, kernel_size, stride
        self.conv2 = nn.Conv2d(64, 128, 3, 1)
        self.dropout = nn.Dropout()
 
        # w*h*output
        self.fc1 = nn.Linear(9*9*128, 1000)
        self.fc2 = nn.Linear(1000, 82)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.dropout(x)
        x = x.view(-1, 9*9*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [121]:
model = Net()
model.to(device)
summary(model, input_size=(1, 45, 45))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 43, 43]             640
            Conv2d-2          [-1, 128, 19, 19]          73,856
           Dropout-3            [-1, 128, 9, 9]               0
            Linear-4                 [-1, 1000]      10,369,000
            Linear-5                   [-1, 82]          82,082
Total params: 10,525,578
Trainable params: 10,525,578
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.34
Params size (MB): 40.15
Estimated Total Size (MB): 41.50
----------------------------------------------------------------


In [122]:
# define the negative log-likelihood loss
loss_func = nn.NLLLoss(reduction='sum')

# define the Adam optimizer
opt = optim.Adam(model.parameters(), lr=1e-4)


In [123]:
# define a helper function to compute the loss value per mini-batch
def loss_batch(loss_func, xb, yb, yb_h, opt=None):
    # obtain loss
    loss = loss_func(yb_h, yb)
    # obtain performance metric
    metric_b = metrics_batch(yb, yb_h)
    if opt is not None:
        loss.backward() # compute gradient
        opt.step() # update parameters
        opt.zero_grad() # set gradients to zero
    return loss.item(), metric_b
    
# define a helper function to compute the accurary per mini-batch
def metrics_batch(target, output):
    # optain output class
    pred = output.argmax(dim=1, keepdim=True)
    # compare output class with target class
    corrects = pred.eq(target.view_as(pred)).sum().item()
    
    return corrects
    
# define a helper fuction to compute the loss and metric values for a dataset
def loss_epoch(model, loss_func, dataset_dl, opt=None):
    loss = 0.0
    metric = 0.0
    len_data = len(dataset_dl.dataset)
    
    for xb, yb in dataset_dl:
        xb = xb.type(torch.float).to(device)
        yb = yb.to(device)
        # obtain model output
        yb_h = model(xb)
        
        loss_b, metric_b = loss_batch(loss_func, xb, yb, yb_h, opt)
        loss += loss_b
        if metric_b is not None:
            metric += metric_b
            
    loss /= len_data
    metric /= len_data
    return loss, metric

In [124]:
# define train_val function
def train_val(epochs, model, loss_func, opt, train_dl, val_dl):
    for epoch in range(epochs):
        model.train() # convert to train mode
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, opt)
        model.eval() # convert to evaluation mode
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl)
        accuracy = 100 * val_metric
        print('epoch: %d, train loss: %.6f, val loss: %.6f, accuracy: %.2f' %(epoch, train_loss, val_loss, accuracy))

In [125]:
train_val(training_epochs, model, loss_func, opt, train_loader, test_loader)

epoch: 0, train loss: 0.968662, val loss: 0.424762, accuracy: 87.79
epoch: 1, train loss: 0.360795, val loss: 0.271629, accuracy: 91.51
epoch: 2, train loss: 0.255513, val loss: 0.211915, accuracy: 93.36
epoch: 3, train loss: 0.198586, val loss: 0.164513, accuracy: 94.41
epoch: 4, train loss: 0.160886, val loss: 0.136845, accuracy: 95.26
epoch: 5, train loss: 0.133085, val loss: 0.113480, accuracy: 96.00
epoch: 6, train loss: 0.112261, val loss: 0.094930, accuracy: 96.52
epoch: 7, train loss: 0.097166, val loss: 0.082924, accuracy: 96.82
epoch: 8, train loss: 0.086027, val loss: 0.074052, accuracy: 97.15
epoch: 9, train loss: 0.076962, val loss: 0.066139, accuracy: 97.35
epoch: 10, train loss: 0.069632, val loss: 0.059491, accuracy: 97.60
epoch: 11, train loss: 0.063712, val loss: 0.056087, accuracy: 97.72
epoch: 12, train loss: 0.058493, val loss: 0.049509, accuracy: 97.88
epoch: 13, train loss: 0.054699, val loss: 0.047802, accuracy: 97.83
epoch: 14, train loss: 0.051870, val loss: 0

In [126]:
torch.save(model.state_dict(), "../model/model_05301.pth")

3 layer
```
epoch: 0, train loss: 1.285788, val loss: 0.696281, accuracy: 81.33
epoch: 1, train loss: 0.551021, val loss: 0.448927, accuracy: 87.15
epoch: 2, train loss: 0.387006, val loss: 0.348044, accuracy: 89.77
epoch: 3, train loss: 0.305232, val loss: 0.288899, accuracy: 91.40
epoch: 4, train loss: 0.253791, val loss: 0.244278, accuracy: 92.54
epoch: 5, train loss: 0.214425, val loss: 0.218080, accuracy: 93.05
epoch: 6, train loss: 0.183755, val loss: 0.185739, accuracy: 94.00
epoch: 7, train loss: 0.158587, val loss: 0.162452, accuracy: 94.76
epoch: 8, train loss: 0.137587, val loss: 0.140578, accuracy: 95.30
epoch: 9, train loss: 0.119347, val loss: 0.126847, accuracy: 95.76
epoch: 10, train loss: 0.104779, val loss: 0.114947, accuracy: 96.02
epoch: 11, train loss: 0.092087, val loss: 0.099882, accuracy: 96.40
epoch: 12, train loss: 0.081314, val loss: 0.089293, accuracy: 96.77
epoch: 13, train loss: 0.071973, val loss: 0.081060, accuracy: 96.97
epoch: 14, train loss: 0.064590, val loss: 0.068942, accuracy: 97.43
```

32 64

```
epoch: 0, train loss: 1.220010, val loss: 0.609274, accuracy: 83.56
epoch: 1, train loss: 0.490901, val loss: 0.396429, accuracy: 88.65
epoch: 2, train loss: 0.351439, val loss: 0.312726, accuracy: 90.60
epoch: 3, train loss: 0.281200, val loss: 0.271936, accuracy: 91.50
epoch: 4, train loss: 0.234941, val loss: 0.236269, accuracy: 92.52
epoch: 5, train loss: 0.199947, val loss: 0.197265, accuracy: 93.72
epoch: 6, train loss: 0.172066, val loss: 0.177574, accuracy: 94.27
epoch: 7, train loss: 0.148420, val loss: 0.157924, accuracy: 94.76
epoch: 8, train loss: 0.129553, val loss: 0.137642, accuracy: 95.29
epoch: 9, train loss: 0.112607, val loss: 0.123169, accuracy: 95.83
epoch: 10, train loss: 0.097524, val loss: 0.112986, accuracy: 96.08
epoch: 11, train loss: 0.085449, val loss: 0.100038, accuracy: 96.45
epoch: 12, train loss: 0.075240, val loss: 0.085687, accuracy: 96.93
epoch: 13, train loss: 0.066709, val loss: 0.073458, accuracy: 97.33
epoch: 14, train loss: 0.060624, val loss: 0.067905, accuracy: 97.42
epoch: 15, train loss: 0.054325, val loss: 0.061316, accuracy: 97.58
epoch: 16, train loss: 0.049916, val loss: 0.057681, accuracy: 97.77
epoch: 17, train loss: 0.046924, val loss: 0.053861, accuracy: 97.84
epoch: 18, train loss: 0.042534, val loss: 0.053883, accuracy: 97.83
epoch: 19, train loss: 0.041439, val loss: 0.051955, accuracy: 97.90
```